| **Author**          | **Roll No**   | **Version** |
|---------------------|----------------|--------------|
| Vinayak Rana        | 24210114       | 1.0          |

#### Introduction
In this notebook we will remove those files that contain bad/unwanted words.

In [12]:
import os
import pandas as pd
import re
import concurrent.futures

In [13]:
source_dict = {}
file_paths = []
src_file_dict = {}

In [14]:
for base_path in (os.getcwd()+'/Data/',os.getcwd()+'/Abhyudaya/Data/',os.getcwd()+'/Shiva/Data/'): 
    
    for item in os.listdir(base_path):
        
        if item =='cc100' or item =='cc100_local' or item =='cc100_local2' or item =='roots_se' or item =='oscar_english_texts' or item=='Oscar_Texts':
            continue

        path = base_path + item
        if item == 'Nptel_data':
            path+='/Pdf text'
        
        total_size = 0
        src_file_dict[item]=os.listdir(path)

        for file in os.listdir(path):
            
            total_size += os.path.getsize(path+'/'+file)
            file_paths.append(path+'/'+file)
        
        source_dict[item] = (total_size/(1024*1024*1024),len(os.listdir(path)))

source_dict, len(file_paths)


({'Nptel_data': (0.7950007421895862, 1143),
  'arxiv_data': (0.13570904824882746, 2703),
  'springer_data': (0.4707957338541746, 1583),
  'imsdb_data': (0.14237183704972267, 1161),
  'gutenberg_data': (5.774677988141775, 15687),
  'authorama_data': (0.057536656036973, 159),
  'textfilesdottxt': (0.01050278078764677, 64),
  'pdrdotorg': (0.021409809589385986, 6908),
  'bbc': (0.026513222604990005, 6479),
  'cnn': (0.12292665150016546, 19078),
  'textfilesdotcom': (1.2625173665583134, 45072),
  'propublicadotorg': (0.18309276923537254, 17652),
  'NLP_Assignment-1': (1.1444091796875e-05, 3),
  'Project_Gutenberg_Data': (12.572358540259302, 34283),
  'Subtitles': (0.019428216852247715, 420),
  'Oscar_articles': (0.000583648681640625, 1)},
 152396)

In [6]:
def create_bad_word_regex(bad_words):
    combined_pattern = '|'.join(re.escape(word.lower()) for word in bad_words)
    regex = r'\b(' + combined_pattern + r')\b'
    return re.compile(regex, re.IGNORECASE)

with open("/mnt/HDFS1/language_nlp/english_nlp_team17/Assignment1/Shiva/Scripts/Bad_words.txt", "r") as file:
    badwords_set = set(line.strip().lower() for line in file)

bad_word_regex = create_bad_word_regex(badwords_set)

In [7]:
def contains_bad_words(text, bad_word_regex):
    return bool(bad_word_regex.search(text))

def process_file(fp, bad_word_regex):
    try:
        with open(fp, 'r', encoding='utf-8') as file:
            content = file.read()
            return fp, contains_bad_words(content, bad_word_regex)  
    except Exception as e:
        return fp, True  

In [13]:
results = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    
    futures = {executor.submit(process_file, fp, bad_word_regex): fp for fp in file_paths}
    
    for future in concurrent.futures.as_completed(futures):
        file_path, result = future.result()  
        results.append((file_path, result))  

results.sort(key=lambda x: file_paths.index(x[0]))  

mask = [result for _, result in results]  

print(mask)


[False, False, False, False, False, False, False, True, False, True, True, False, False, False, False, False, True, True, True, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, True, True, False, True, False, False, False, False, False, False, True, False, True, False, False, False, True, False, False, False, True, True, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, True, True, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, True, False, True, False, False, True, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, True, False, True, False, True, F

In [17]:
sum(mask), len(file_paths)

(58914, 152393)

In [23]:
good_file_paths = [fp for fp, is_bad in zip(file_paths, mask) if not is_bad]

In [24]:
len(good_file_paths)

93479

In [32]:
good_files = [os.path.basename(path) for path in good_file_paths]

In [38]:
with open('good_files', 'w') as file:
    for path in good_file_paths:
        file.write(path + '\n')

In [35]:
cleaned_src_file_dict = {source: [file for file in files if file in good_files]for source, files in src_file_dict.items()}

In [40]:
num_files = [len(files) for src,files in cleaned_src_file_dict.items()]

In [50]:
clean_dict = {key:0 for key in cleaned_src_file_dict}

In [63]:
for path in good_file_paths:

    folder_path = os.path.dirname(path)  
    extracted_folder = os.path.basename(folder_path)  

    if extracted_folder == 'Pdf text':
        clean_dict['Nptel_data']+=os.path.getsize(path)
        continue

    clean_dict[extracted_folder]+=os.path.getsize(path)

temp={key:value/(1024*1024*1024) for key,value in clean_dict.items()}
temp = dict(filter(lambda item: item[0] != 'Oscar_articles', temp.items()))
temp

{'Nptel_data': 0.8894380070269108,
 'arxiv_data': 0.261197617277503,
 'springer_data': 0.26697950810194016,
 'imsdb_data': 0.01065288856625557,
 'gutenberg_data': 0.9501577448099852,
 'authorama_data': 0.014744412153959274,
 'textfilesdottxt': 0.004979614168405533,
 'pdrdotorg': 0.033383406698703766,
 'bbc': 0.04844621755182743,
 'cnn': 0.22902477905154228,
 'textfilesdotcom': 0.8981762491166592,
 'propublicadotorg': 0.30709875002503395,
 'Project_Gutenberg_Data': 2.158886142075062,
 'Subtitles': 0.006696891039609909}

In [67]:
data = {
    'Volume (in GB)': list(temp.values()),
    'Number of Files': num_files[:-1]  
}

df = pd.DataFrame(data, index=list(temp.keys()))

In [68]:
# Cleaned data info
df

,Volume (in GB),Number of Files
Nptel_data,0.889438,1142
arxiv_data,0.261198,2637
springer_data,0.266980,1059
imsdb_data,0.010653,61
gutenberg_data,0.950158,9355
authorama_data,0.014744,34
textfilesdottxt,0.004980,32
pdrdotorg,0.033383,6427
bbc,0.048446,6474
cnn,0.229025,18127


In [27]:
# Original Data info

df_original = pd.DataFrame(source_dict).transpose()
df_original.columns=['Volume(in GB)','Number of files']
df_original['Number of files'] = df_original['Number of files'].round().astype('int')

df_original = df_original.drop('NLP_Assignment-1')
df_original = df_original.drop('Oscar_articles')

df_original

,Volume(in GB),Number of files
Nptel_data,0.795001,1143
arxiv_data,0.135709,2703
springer_data,0.470796,1583
imsdb_data,0.142372,1161
gutenberg_data,5.774678,15687
authorama_data,0.057537,159
textfilesdottxt,0.010503,64
pdrdotorg,0.021410,6908
bbc,0.026513,6479
cnn,0.122927,19078


#### THANK YOU